**This notebook is for:**
1. model population.
2. Push variables (recency and number of times in the last 7 days that an user was sended a push notification)
3. Generte the most important variables from the previous model.

In [1]:
import sys, os 
prj_path = os.path.dirname(os.getcwd())
utils_path = prj_path + '/utils'
if  not utils_path in sys.path:
    print('adding utils to path ')
    sys.path.insert(1, utils_path)

adding utils to path 


In [2]:
from utilities import *

from google_cloud import BigQuery, Storage
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime

import os
import sys

import boto3
import base64

#Cliente BigQuery
AUTH_BIGQUERY = base64.b64decode(os.environ['SECRET_AUTH_BIGQUERY_MODEL'])
bq = BigQuery(AUTH_BIGQUERY)
storage =Storage(AUTH_BIGQUERY)

#Load config.
files = glob.glob(f"{utils_path}/*.yml") 
conf = load_conf(files)

In [3]:
site = conf['SITE']
backbone_table = conf['BACKBONE_TABLE_NAME']
time = ('2022-01-05', '2022-03-21')

In [4]:
%%time
#Backbone (push<-orders = target)
backbone_table_production(table_name = backbone_table,
                              site = site,
                              conf = conf,
                              periods = time
                             )

2022-03-22 08:07:20 Create table meli-marketing.TEMP45.CY_NB_ML_BACKBONE_MLB if not exist
2022-03-22 08:07:20 This service is instrumented using OpenTelemetry.OpenTelemetry could not be imported; pleaseadd opentelemetry-api and opentelemetry-instrumentationpackages in order to get BigQuery Tracing data.
2022-03-22 08:07:21 Truncate table meli-marketing.TEMP45.CY_NB_ML_BACKBONE_MLB
2022-03-22 08:07:22 Backbone for New Buyers: from 2022-01-05 to 2022-03-21
2022-03-22 08:08:26 End Backbone building meli-marketing.TEMP45.CY_NB_ML_BACKBONE_MLB


CPU times: user 250 ms, sys: 61.6 ms, total: 311 ms
Wall time: 1min 6s


In [4]:
%%time
#Push var
Var2GCP(variable = 'PUSH_VARS' , 
        conf = conf, 
        query_ = 'PUSH_LAST_7_DAYS', 
        query_param = {'backbone':backbone_table,'site':site},
        type_= 'train'
       )

2022-03-22 09:01:27 Start creating variable : PUSH_VARS for site MLB
2022-03-22 09:01:27 Writing on : gs://marketing-modelling/ML/NB/MLB/train/dataset/PUSH_VARS
2022-03-22 09:01:27 Excecuting PUSH_LAST_7_DAYS with params: {'backbone': 'meli-marketing.TEMP45.CY_NB_ML_BACKBONE', 'site': 'MLB'}
2022-03-22 09:01:28 This service is instrumented using OpenTelemetry.OpenTelemetry could not be imported; pleaseadd opentelemetry-api and opentelemetry-instrumentationpackages in order to get BigQuery Tracing data.


Query results loaded to the table meli-marketing.TEMP45.lnapsxtruv


2022-03-22 09:11:07 End process for variable : PUSH_VARS


Exporto meli-marketing:TEMP45.lnapsxtruv a gs://marketing-modelling/ML/NB/MLB/train/dataset/PUSH_VARS/*.parquet
Deleted table 'meli-marketing.TEMP45.lnapsxtruv'.
CPU times: user 411 ms, sys: 56.1 ms, total: 467 ms
Wall time: 9min 40s


In [5]:
%%time
#VARS FROM THE TOP VARIABLES (LAST MODEL)
Var2GCP(variable = 'BEHAVIOUR_VARS' , 
            conf = conf, 
            query_ = 'BEHAVIOUR_VARIABLES', 
            query_param = {'backbone':backbone_table,'site':site},
            type_= 'train',
           )

2022-03-22 09:40:48 Start creating variable : BEHAVIOUR_VARS for site MLB
2022-03-22 09:40:48 Writing on : gs://marketing-modelling/ML/NB/MLB/train/dataset/BEHAVIOUR_VARS
2022-03-22 09:40:48 Excecuting BEHAVIOUR_VARIABLES with params: {'backbone': 'meli-marketing.TEMP45.CY_NB_ML_BACKBONE', 'site': 'MLB'}


Query results loaded to the table meli-marketing.TEMP45.xkccfjrmbe


2022-03-22 09:58:39 End process for variable : BEHAVIOUR_VARS


Exporto meli-marketing:TEMP45.xkccfjrmbe a gs://marketing-modelling/ML/NB/MLB/train/dataset/BEHAVIOUR_VARS/*.parquet
Deleted table 'meli-marketing.TEMP45.xkccfjrmbe'.
CPU times: user 248 ms, sys: 23.2 ms, total: 271 ms
Wall time: 17min 51s


In [ ]:
for i,x,z in zip(['PUSH_VARS','BEHAVIOUR_VARS'],['PUSH_LAST_7_DAYS','BEHAVIOUR_VARIABLES'],[{'backbone':backbone_table,'site':site},{'backbone':backbone_table,'site':site}]):
    print(i,x,z)

In [ ]:
varname = ['PUSH_VARS','BEHAVIOUR_VARS']
queries = ['PUSH_LAST_7_DAYS','BEHAVIOUR_VARIABLES']
parameters = [{'backbone':backbone_table,'site':site},
              {'backbone':backbone_table,'site':site}]

for v,q,qp in zip(varname,queries,parameters):
    Var2GCP(variable = v ,
            conf = conf, 
            query_ = q, 
            query_param = qp,
            type_= 'train')

In [ ]:
#GAIA:
#JOBS: HYPER - V_BT_PUSH_COUPON_CAMPAIGN_EVENT (genera el extracto para el dash)
#      PUSH - BT_PUSH_ML_BATCH_COUPON_CAMPAIGN_EVENT

# meli-marketing.MKTPUBLIC.V_BT_PUSH_NOTIFICATION_EVENT
# "BATCH_ID": "MLB_MP_PUSHMP_AO_P_ALL_RECU_SELL_TRANSFERENCIA-DEVICE_DEFAULT",
# "CAMPAIGN_ID": "MLB_MP_PUSHMP_AO_P_ALL_RECU_SELL_TRANSFERENCIA-DEVICE",
# "CAMPAIGN_TYPE": "recurrente_mp",
# "SENT_DATE": "2021-02-11",
# "APP": "mercadopago",
# "SIT_SITE_ID": "MLB",
# "TIM_DAY": "2021-02-11",
# "CUS_CUST_ID": "186910412",
# "EVENT_TYPE": "arrived",
# "DEVICE_PLATFORM": "/mobile/ios",
# "ADVERTISER_ID": "EC3F9C41-01FE-4E58-8A70-B28FE7C658F4"

#EVENT_TYPE: {holdre,holdad,sent,shown,open,control,blacklist,discarded,arrived}
#SENT_DATE: {min:2022-01-05, max:2022-02-22}

#Measures (tableau).
#Exposure: SHOW/SENT
#Open Rate: OPEN/SHOWN (check con Pedro, valores no cuadran con el dash)

In [ ]:
%%time
send_push_groups = bq.execute_response("""
SELECT push.EVENT_TYPE,push.SENT_DATE, count(distinct push.CUS_CUST_ID)
FROM meli-marketing.MKTPUBLIC.V_BT_PUSH_NOTIFICATION_EVENT push 
WHERE BATCH_ID LIKE 'MLB_ML_PUSHML_AO_ALL_X_ACT_ALL_NEWBUYERS%' 
    AND push.APP = 'mercadolibre' 
    AND push.sit_site_id = 'MLB' 
    AND push.SENT_DATE >= DATE '2022-01-01'
GROUP BY 1,2
""")

In [ ]:
send_push_groups['SENT_DATE'] = pd.to_datetime(send_push_groups.SENT_DATE)

send_push_groups_pivot = send_push_groups.pivot_table(index = 'SENT_DATE', values = 'f0_',columns = 'EVENT_TYPE').reset_index()
send_push_groups_pivot['open_rate'] = send_push_groups_pivot.open/(send_push_groups_pivot.shown)
send_push_groups_pivot['exposure'] = send_push_groups_pivot.shown/(send_push_groups_pivot.sent)

In [ ]:
#SEND
send_push_groups_pivot.plot.bar(x = 'SENT_DATE',y = 'sent', figsize = (16,5))
plt.grid()

In [ ]:
send_push_groups_pivot['weekday'] = send_push_groups_pivot.SENT_DATE.dt.weekday

In [ ]:
#HAy variaciones en la cantidad de push enviadas por dia de la semana?
send_push_groups_pivot.boxplot(column = 'sent',by = 'weekday')
#0: Lunes
#6: domingo

In [ ]:
#OPEN RATE
send_push_groups_pivot.plot.bar(x = 'SENT_DATE',y = 'open_rate', figsize = (16,5))
plt.grid()
#Parece uniforme el open rate 

In [ ]:
#Se diferencia el open rate por dia de la semana?
send_push_groups_pivot.boxplot(column = 'open_rate',by = 'weekday')

In [ ]:
send_push_groups_pivot.groupby('weekday').sent.sum().sum()

In [ ]:
#Cantidades nominales
(send_push_groups_pivot.groupby('weekday').sent.sum()*100)/send_push_groups_pivot.groupby('weekday').sent.sum().sum()

In [ ]:
# CONVERSION
site = 'MLB'
date_intial = '2022-01-01'
date_end = '2022-01-07'
conversion_query = \
f"""
    WITH push as ( 
        SELECT push.CUS_CUST_ID,push.sit_site_id, push.SENT_DATE, push.EVENT_TYPE 
        FROM meli-marketing.MKTPUBLIC.V_BT_PUSH_NOTIFICATION_EVENT push 
        WHERE BATCH_ID LIKE 'MLB_ML_PUSHML_AO_ALL_X_ACT_ALL_NEWBUYERS%' 
        AND push.EVENT_TYPE in ('sent','control') 
        AND push.APP = 'mercadolibre'
        AND push.sit_site_id = '{site}' 
        AND push.SENT_DATE BETWEEN DATE '{date_intial}' AND DATE '{date_end}' 
         ),
    orders as (
    SELECT push.CUS_CUST_ID, 
    push.SENT_DATE, 
    push.EVENT_TYPE,
    orders.ORD_CREATED_DT,
    ROW_NUMBER() OVER (PARTITION BY push.CUS_CUST_ID ORDER BY orders.ORD_CREATED_DT) AS ORDERING_PURCHASE
    FROM push LEFT JOIN meli-bi-data.WHOWNER.BT_ORD_ORDERS orders
    ON push.CUS_CUST_ID = orders.ORD_BUYER.ID 
        AND push.sit_site_id = orders.SIT_SITE_ID 
        AND ORD_CLOSED_DT is not null
        AND orders.ord_status = 'paid' 
        AND orders.ORD_CLOSED_DT BETWEEN push.SENT_DATE AND DATE_ADD(push.SENT_DATE, INTERVAL 4 DAY) 
        AND orders.ORD_CREATED_DT = push.SENT_DATE
    )
    
    SELECT * EXCEPT(ORDERING_PURCHASE), CASE WHEN ORD_CREATED_DT is null THEN 0 ELSE 1 END AS CONVERSION
    FROM orders WHERE ORDERING_PURCHASE = 1
""" 
conversion  = " ".join(conversion_query.split()) 
conversion

In [ ]:
%%time
conversion_df = bq.execute_response(conversion)

In [ ]:
conversion_df.head()

In [ ]:
np.round(conversion_df.groupby(['SENT_DATE','EVENT_TYPE']).CONVERSION.mean()*100,3)

In [ ]:
# CONVERSION
site = 'MLB'
date_intial = '2022-01-01'
date_end = '2022-01-07'
conversion_query = \
f"""
    WITH push as ( 
        SELECT push.CUS_CUST_ID,push.sit_site_id, push.SENT_DATE, push.EVENT_TYPE 
        FROM meli-marketing.MKTPUBLIC.V_BT_PUSH_NOTIFICATION_EVENT push 
        WHERE BATCH_ID LIKE 'MLB_ML_PUSHML_AO_ALL_X_ACT_ALL_NEWBUYERS%' 
        AND push.EVENT_TYPE in ('sent','control') 
        AND push.APP = 'mercadolibre'
        AND push.sit_site_id = '{site}' 
        AND push.SENT_DATE BETWEEN DATE '{date_intial}' AND DATE '{date_end}' 
         ),
    orders as (
    SELECT push.CUS_CUST_ID, 
    push.SENT_DATE, 
    push.EVENT_TYPE,
    orders.ORD_CREATED_DT,
    ROW_NUMBER() OVER (PARTITION BY push.CUS_CUST_ID ORDER BY orders.ORD_CREATED_DT) AS ORDERING_PURCHASE
    FROM push LEFT JOIN meli-bi-data.WHOWNER.BT_ORD_ORDERS orders
    ON push.CUS_CUST_ID = orders.ORD_BUYER.ID 
        AND push.sit_site_id = orders.SIT_SITE_ID 
        AND ORD_CLOSED_DT is not null
        AND orders.ord_status = 'paid' 
        AND orders.ORD_CLOSED_DT BETWEEN push.SENT_DATE AND DATE_ADD(push.SENT_DATE, INTERVAL 4 DAY) 
        AND orders.ORD_CREATED_DT = push.SENT_DATE
    )
    
    SELECT * EXCEPT(ORDERING_PURCHASE), CASE WHEN ORD_CREATED_DT is null THEN 0 ELSE 1 END AS CONVERSION
    FROM orders WHERE ORDERING_PURCHASE = 1
""" 
conversion  = " ".join(conversion_query.split()) 
conversion

In [ ]:
WITH SENTS AS(
    select b.cus_cust_id,b.sit_site_id,SENT_DATE,EVENT_TYPE
    from meli-marketing.MKTPUBLIC.V_BT_PUSH_NOTIFICATION_EVENT b
    where BATCH_ID like 'MLB_ML_PUSHML_AO_ALL_X_ACT_ALL_NEWBUYERS%'
        and b.EVENT_TYPE in ('sent','control')
        and APP = 'mercadolibre'
        and sit_site_id = 'MLB'
        AND b.SENT_DATE between DATE'2022-02-12' AND DATE'2022-02-14'
    group by 1,2,3,4
    )
    select  s.cus_cust_id,EVENT_TYPE, sent_date,
            buy_proba,
        MAX(CASE WHEN ords.ORD_BUYER.ID is not null THEN 1 ELSE 0 END) as buy
    from  SENTS s
    left join meli-marketing.MODELLING.NB_PREDICT_PROBA nb
        on nb.cus_cust_id = s.cus_cust_id
        AND nb.process_date = s.sent_date
        AND nb.sit_site_id = s.sit_site_id
    left join meli-bi-data.WHOWNER.BT_ORD_ORDERS ords
        on s.sit_site_id = ords.sit_site_id
        and s.cus_cust_id = ords.ORD_BUYER.ID
        AND ORD_STATUS = 'paid' and ORD_CLOSED_DT is not null
        AND ORD_CLOSED_DT between sent_date AND sent_date+4
        AND ORD_CREATED_DT = sent_date
    group by 1,2,3,4

In [ ]:
#Pagos con descuento.
WITH TOOLS as(
SELECT  CAST(MKT_TOOL_ID as INT64) as CAMPAIGN_ID, sit_site_id
           FROM `meli-marketing.PUSH.LK_PUSH_BUDGET_COUPON` cp
           WHERE cp.sit_site_id in ('MLB')
        GROUP BY 1,2
)
  SELECT  SUM(cpn.mkt_cpn_amount) as descuento_lc
  FROM  `meli-bi-data.WHOWNER.BT_PAY_PAYMENTS` as pay
  INNER JOIN `meli-bi-data.WHOWNER.BT_MKT_COUPON_V2` as cpn
      on pay.pay_payment_id = cpn.pay_payment_id
      and pay.cus_cust_id_buy = cpn.cus_cust_id_buy
  INNER JOIN TOOLS t
    ON t.CAMPAIGN_ID = mkt_cpn_campaign_id
    AND pay.SIT_SITE_ID = t.SIT_SITE_ID
  WHERE
      pay.pay_status_code = 'approved'
      AND pay.SIT_SITE_ID in ('MLB')
      AND pay.pay_created_dt = date'2022-02-14'

In [ ]:
#Tabla de push.
#Comparar con converson.

In [ ]:
new_buyer = pd.concat(new_buyer_list)
new_buyer['CUS_RU_SINCE'] = pd.to_datetime(new_buyer.CUS_RU_SINCE)

In [ ]:
# %%time
# new_buyer = bq.execute_response(new_buyer_query)
# new_buyer['CUS_RU_SINCE'] = pd.to_datetime(new_buyer.CUS_RU_SINCE)

In [ ]:
new_buyer.info()

In [ ]:
new_buyer.groupby('CUS_RU_SINCE').CUS_CUST_ID.count().plot.line(figsize = (14,6))